# Laboratorio: Modelos del lenguaje con RNNs

En este laboratorio, vamos a entrenar un modelo del lenguaje basado en caracteres con Recurrent Neural Networks. Asimismo, utilizaremos el modelo para generar texto. En particular, alimentaremos nuestro modelo con obras de la literatura clásica en castellano para obtener una red neuronal que sea capaz de "escribir" fragmentos literarios.

Los entrenamientos en esta laboratorio para obtener un modelo de calidad podrían tomar cierto tiempo (5-10 minutos por epoch), por lo que se aconseja empezar a trabajar pronto. El uso de GPUs no ayuda tanto con LSTMs como con CNNs, por lo que si tenéis máquinas potentes en casa es posible que podáis entrenar más rápido o a la misma velocidad que en Colab. En todo caso, la potencia de Colab es más que suficiente para completar este laboratorio con éxito.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/d/d8/El_ingenioso_hidalgo_don_Quijote_de_la_Mancha.jpg" style="text-align: center" height="300px"></center>

El dataset a utilizar consistirá en un archivo de texto con el contenido íntegro en castellano antiguo de El Ingenioso Hidalgo Don Quijote de la Mancha, disponible de manera libre en la página de [Project Gutenberg](https://www.gutenberg.org). Asimismo, como apartado optativo en este laboratorio se pueden utilizar otras fuentes de texto. Aquí podéis descargar los datos a utilizar de El Quijote y un par de obras adicionales:

[El ingenioso hidalgo Don Quijote de la Mancha (Miguel de Cervantes)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io)

[Compilación de obras teatrales (Calderón de la Barca)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219433&authkey=AKvGD6DC3IRBqmc)

[Trafalgar (Benito Pérez Galdós)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ)

Como ya deberíamos de estar acostumbrados en problemas de Machine Learning, es importante echar un vistazo a los datos antes de empezar.

## 1. Carga y procesado del texto

Primero, vamos a descargar el libro e inspeccionar los datos. El fichero a descargar es una versión en .txt del libro de Don Quijote, a la cual se le han borrado introducciones, licencias y otras secciones para dejarlo con el contenido real de la novela.

In [1]:
import numpy as np 
import keras
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import random
import io

import os, sys

path = keras.utils.get_file(
    fname="don_quijote.txt", 
    origin="https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io"
)

Using TensorFlow backend.


Una vez descargado, vamos a leer el contenido del fichero en una variable. Adicionalmente, convertiremos el contenido del texto a minúsculas para ponérselo un poco más fácil a nuestro modelo (de modo que todas las letras sean minúsculas y el modelo no necesite diferenciar entre minúsculas y mayúsculas).

**1.1.** Leer todo el contenido del fichero en una única variable ***text*** y convertir el string a minúsculas

In [2]:
if os.name == 'nt':
    file_path = os.path.join(r'C:\Users\manuel\.keras\datasets', 'don_quijote.txt')
else:
    file_path = os.path.join('~/.keras/datasets', 'don_quijote.txt')
    
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

# convierto a minúsculas
text = text.lower()

Podemos comprobar ahora que efectivamente nuestra variable contiene el resultado deseado, con el comienzo tan característico del Quijote.

In [3]:
print("Longitud del texto: {}".format(len(text)))
print(text[0:300])

Longitud del texto: 2071198
capítulo primero. que trata de la condición y ejercicio del famoso hidalgo
don quijote de la mancha


en un lugar de la mancha, de cuyo nombre no quiero acordarme, no ha mucho
tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua,
rocín flaco y galgo corredor. una olla de algo más


## 2. Procesado de los datos

Una de las grandes ventajas de trabajar con modelos que utilizan caracteres en vez de palabras es que no necesitamos tokenizar el texto (partirlo palabra a palabra). Nuestro modelo funcionará directamente con los caracteres en el texto, incluyendo espacios, saltos de línea, etc.

Antes de hacer nada, necesitamos procesar el texto en entradas y salidas compatibles con nuestro modelo. Como sabemos, un modelo del lenguaje con RNNs acepta una serie de caracteres y predice el siguiente carácter en la secuencia.

* "*El ingenioso don Qui*" -> predicción: **j**
* "*El ingenioso don Quij*" -> predicción: **o**

De modo que la entrada y la salida de nuestro modelo necesita ser algo parecido a este esquema. En este punto, podríamos usar dos formas de preparar los datos para nuestro modelo.

1. **Secuencia a secuencia**. La entrada de nuestro modelo sería una secuencia y la salida sería esa secuencia trasladada un caracter a la derecha, de modo que en cada instante de tiempo la RNN tiene que predecir el carácter siguiente. Por ejemplo:

>* *Input*:   El ingenioso don Quijot 
>* *Output*: l ingenioso don Quijote

2. **Secuencia a carácter**. En este variante, pasaríamos una secuencia de caracteres por nuestra RNN y, al llegar al final de la secuencia, predeciríamos el siguiente carácter.

>* *Input*:   El ingenioso don Quijot 
>* *Output*: e

En este laboratorio, por simplicidad, vamos a utilizar la segunda variante.

De este modo, a partir del texto, hemos de generar nuestro propio training data que consista en secuencias de caracteres con el siguiente carácter a predecir. Para estandarizar las cosas, utilizaremos secuencias de tamaño *SEQ_LENGTH* caracteres (un hiperparámetro que podemos elegir nosotros).



#### 2.1. Obtención de los caracteres y mapas de caracteres

Antes que nada, necesitamos saber qué caracteres aparecen en el texto, ya que tendremos que diferenciarlos mediante un índice de 0 a *num_chars* - 1 en el modelo. Obtener:
 

1.   Número de caracteres únicos que aparecen en el texto.
2.   Diccionario que asocia char a índice único entre 0 y *num_chars* - 1. Por ejemplo, {'a': 0, 'b': 1, ...}
3.   Diccionario reverso de índices a caracteres: {0: 'a', 1: 'b', ...}


In [4]:
# Busco los caracteres:
caracter_v = []
for c in text:
    if c not in caracter_v:
        caracter_v.append(c)
caracter_v.sort()

print(' - Se encontraron {:d} caracteres: {}'.format(len(caracter_v), caracter_v) )


# Armamos los diccionarios correspondientes.
idx_v = list(range(len(caracter_v)))
chr2idx_d = dict(zip(caracter_v, idx_v))
idx2chr_d = dict(zip(idx_v, caracter_v))

print(' - chr2idx_d = ', chr2idx_d)
print(' - idx2chr_d = ', idx2chr_d)

 - Se encontraron 61 caracteres: ['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', ':', ';', '?', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', '«', '»', '¿', 'à', 'á', 'é', 'í', 'ï', 'ñ', 'ó', 'ù', 'ú', 'ü']
 - chr2idx_d =  {'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, ':': 18, ';': 19, '?': 20, ']': 21, 'a': 22, 'b': 23, 'c': 24, 'd': 25, 'e': 26, 'f': 27, 'g': 28, 'h': 29, 'i': 30, 'j': 31, 'l': 32, 'm': 33, 'n': 34, 'o': 35, 'p': 36, 'q': 37, 'r': 38, 's': 39, 't': 40, 'u': 41, 'v': 42, 'w': 43, 'x': 44, 'y': 45, 'z': 46, '¡': 47, '«': 48, '»': 49, '¿': 50, 'à': 51, 'á': 52, 'é': 53, 'í': 54, 'ï': 55, 'ñ': 56, 'ó': 57, 'ù': 58, 'ú': 59, 'ü': 60}
 - idx2chr_d =  {0: '\n', 1: ' ', 2: '!', 3: '"', 4: "'", 5: '(', 6: ')', 7: ',', 8: '-', 9: '.', 10: '

#### 2.2. Obtención de secuencias de entrada y carácter a predecir

Ahora, vamos a obtener las secuencias de entrada en formato texto y los correspondientes caracteres a predecir. Para ello, recorrer el texto completo leído anteriormente, obteniendo una secuencia de SEQ_LENGTH caracteres y el siguiente caracter a predecir. Una vez hecho, desplazarse un carácter a la izquierda y hacer lo mismo para obtener una nueva secuencia y predicción. Guardar las secuencias en una variable ***sequences*** y los caracteres a predecir en una variable ***next_chars***.

Por ejemplo, si el texto fuera "Don Quijote" y SEQ_LENGTH fuese 5, tendríamos

* *sequences* = ["Don Q", "on Qu", "n Qui", " Quij", "Quijo", "uijot"]
* *next_chars* = ['u', 'i', 'j', 'o', 't', 'e']

In [5]:
# Definimos el tamaño de las secuencias. Puedes dejar este valor por defecto.
SEQ_LENGTH = 30

sequences = []
next_chars = []

for i in range(0, len(text) - SEQ_LENGTH - 1):
    sequences.append(  text[i:i+SEQ_LENGTH] )
    next_chars.append( text[i+SEQ_LENGTH]   )
    
assert len(sequences) == len(next_chars), 'ERROR, armaste mal las sequencias'

Indicar el tamaño del training set que acabamos de generar.

In [6]:
print(' Cantidad de secuencias armadas = ', len(sequences))

 Cantidad de secuencias armadas =  2071167


Como el Quijote es muy largo y tenemos muchas secuencias, podríamos encontrar problemas de memoria. Por ello, vamos a elegir un número máximo de ellas. Si estás corriendo esto localmente y tienes problemas de memoria, puedes reducir el tamaño aún más, pero ten cuidado porque, a menos datos, peor calidad del modelo.

In [7]:
MAX_SEQUENCES = 50000

perm = np.random.permutation(len(sequences))
sequences, next_chars = np.array(sequences), np.array(next_chars)
sequences, next_chars = sequences[perm], next_chars[perm]
sequences, next_chars = list(sequences[:MAX_SEQUENCES]), list(next_chars[:MAX_SEQUENCES])

print(len(sequences))

50000


#### 2.3. Obtención de input X y output y para el modelo

Finalmente, a partir de los datos de entrenamiento que hemos generado vamos a crear los arrays de datos X e y que pasaremos a nuestro modelo.

Para ello, vamos a utilizar *one-hot encoding* para nuestros caracteres. Por ejemplo, si sólo tuviéramos 4 caracteres (a, b, c, d), las representaciones serían: (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0) y (0, 0, 0, 1).

De este modo, **X** tendrá shape *(num_sequences, seq_length, num_chars)* e **y** tendrá shape *(num_sequences, num_chars)*. 



In [8]:
NUM_CHARS = len(chr2idx_d)
NUM_SEQUENCES = len(sequences)
X = np.zeros((NUM_SEQUENCES, SEQ_LENGTH, NUM_CHARS), dtype=np.int8)
y = np.zeros((NUM_SEQUENCES, NUM_CHARS),             dtype=np.int8)

for i_s in range(NUM_SEQUENCES):
    for i_c in range(SEQ_LENGTH):
        X[i_s, i_c, chr2idx_d[sequences[i_s][i_c]]] = 1.0
    
    y[i_s, chr2idx_d[next_chars[i_s]]] = 1.0

assert all([(np.array( [chr2idx_d[a]  for a in sequences[i_s]] ) == np.argmax(X[i_s], axis=-1)).all() for i_s in range(0, NUM_SEQUENCES, NUM_SEQUENCES//100)]), 'ERROR, armaste mal X'

## 3. Definición del modelo y entrenamiento

Una vez tenemos ya todo preparado, es hora de definir el modelo. Define un modelo que utilice una **LSTM** con **128 unidades internas**. Si bien el modelo puede definirse de una manera más compleja, para empezar debería bastar con una LSTM más una capa Dense con el *softmax* que predice el siguiente caracter a producir. Adam puede ser una buena elección de optimizador.

Una vez el modelo esté definido, entrénalo un poco para asegurarte de que la loss es decreciente. No es necesario guardar la salida de este entrenamiento en el entregable final, ya que vamos a hacer el entrenamiento más informativo en el siguiente punto.

In [9]:
def build_model(input_shape, n_output=61, lr=1.e-3):
    model = keras.Sequential()
    model.add( keras.layers.InputLayer(input_shape=input_shape))
    model.add( keras.layers.LSTM(128, return_sequences=False))
    
#     model.add( keras.layers.Flatten() )
    model.add( keras.layers.Dense(n_output, activation='softmax') )
    
    opt = keras.optimizers.Adam(lr=lr)
    model.compile(opt, 'categorical_crossentropy', metrics=['categorical_accuracy'])
    
    model.summary()
    return model
                  
input_shape = X.shape[1:]
n_output    = y.shape[-1]

model = build_model(input_shape, n_output, lr=1e-3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               97280     
_________________________________________________________________
dense_1 (Dense)              (None, 61)                7869      
Total params: 105,149
Trainable params: 105,149
Non-trainable params: 0
_________________________________________________________________


Para ver cómo evoluciona nuestro modelo del lenguaje, vamos a generar texto según va entrenando. Para ello, vamos a programar una función que, utilizando el modelo en su estado actual, genere texto, con la idea de ver cómo se va generando texto al entrenar cada epoch.

En el código de abajo podemos ver una función auxiliar para obtener valores de una distribución multinomial. Esta función se usará para muestrear el siguiente carácter a utilizar según las probabilidades de la salida de softmax (en vez de tomar directamente el valor con la máxima probabilidad, obtenemos un valor aleatorio según la distribución de probabilidad dada por softmax, de modo que nuestros resultados serán más diversos, pero seguirán teniendo "sentido" ya que el modelo tenderá a seleccionar valores con más probabilidad).



In [10]:
def sample(probs, temperature=1.0):
    """Nos da el índice del elemento a elegir según la distribución
    de probabilidad dada por probs.
    
    Args:
      probs es la salida dada por una capa softmax:
        probs = model.predict(x_to_predict)[0]
      
      temperature es un parámetro que nos permite obtener mayor
        "diversidad" a la hora de obtener resultados. 
        
        temperature = 1 nos da la distribución normal de softmax
        0 < temperature < 1 hace que el sampling sea más conservador,
          de modo que sampleamos cosas de las que estamos más seguros
        temperature > 1 hace que los samplings sean más atrevidos,
          eligiendo en más ocasiones clases con baja probabilidad.
          Con esto, tenemos mayor diversidad pero se cometen más
          errores.
    """
    # Cast a float64 por motivos numéricos
    probs = np.asarray(probs).astype('float64')
    
    # Hacemos logaritmo de probabilidades y aplicamos reducción
    # por temperatura.
    probs = np.log(probs) / temperature
    
    # Volvemos a aplicar exponencial y normalizamos de nuevo
    exp_probs = np.exp(probs)
    probs = exp_probs / np.sum(exp_probs)
    
    # Hacemos el sampling dadas las nuevas probabilidades
    # de salida (ver doc. de np.random.multinomial)
    samples = np.random.multinomial(1, probs, 1)
    return np.argmax(samples)


Utilizando la función anterior y el modelo entrenado, vamos a añadir un callback a nuestro modelo para que, según vaya entrenando, veamos los valores que resultan de generar textos con distintas temperaturas al acabar cada epoch.

Para ello, abajo tenéis disponible el callback *on_epoch_end*. Esta función elige una secuencia de texto al azar en el texto disponible en la variable
text y genera textos de longitud *GENERATED_TEXT_LENGTH* según las temperaturas en *TEMPERATURES_TO_TRY*, utilizando para ello la función *generate_text*.

Completa la función *generate_text* de modo que utilicemos el modelo y la función sample para generar texto.

NOTA: Cuando hagas model.predict, es aconsejable usar verbose=0 como argumento para evitar que la función imprima valores de salida.

In [76]:
s = sequences[30]
print(s)
print( generate_text(s, model, length=200, temperature=1) )

as, y tú debes de haber
pensad
as, y tú debes de haber
pensado que el se la de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de de


In [20]:
TEMPERATURES_TO_TRY = [0.2, 0.5, 1.0, 1.2]
GENERATED_TEXT_LENGTH = 300
def text_to_seq(seed_text='hola'):
    """Arma una secuencia a partir de un texto ingresado.
    """
    seq = np.zeros( (1, len(seed_text), NUM_CHARS), dtype=np.int8) 
    for i_c, c in enumerate(seed_text):
        seq[0, i_c, chr2idx_d[c]] = 1

    return seq

def generate_text(seed_text, model, length, temperature=1):
    """Genera una secuencia de texto a partir de seed_text utilizando model.
    
    La secuencia tiene longitud length y el sampling se hace con la temperature
    definida.
    """
    
    # Aquí guardaremos nuestro texto generado, que incluirá el
    # texto origen
    generated = seed_text
    
    # Utilizar el modelo en un bucle de manera que generemos
    # carácter a carácter. Habrá que construir los valores de
    # X_pred de manera similar a como hemos hecho arriba, salvo que
    # aquí sólo se necesita una oración
    # Nótese que el x que utilicemos tiene que irse actualizando con
    # los caracteres que se van generando. La secuencia de entrada al
    # modelo tiene que ser una secuencia de tamaño SEQ_LENGTH que
    # incluya el último caracter predicho.
 
    ### TU CÓDIGO AQUÍ

    
    for i in range(length):
        y_pred     = model.predict( text_to_seq(generated[-SEQ_LENGTH:]), verbose=False)
        generated += idx2chr_d[ np.argmax(y_pred, axis=-1)[0] ]
    
    ### FIN DE TU CÓDIGO
    return generated


def on_epoch_end(epoch, logs):
    print("\n\n\n")

    # Primero, seleccionamos una secuencia al azar para empezar a predecir
    # a partir de ella
    start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
    seed_text = text[start_pos:start_pos + SEQ_LENGTH]
    for temperature in TEMPERATURES_TO_TRY:
        print("------> Epoch: {} - Generando texto con temperature {}".format(epoch + 1, temperature))

        generated_text = generate_text(seed_text, model, 
                                       GENERATED_TEXT_LENGTH, temperature)
        print("Seed: {}".format(seed_text))
        print("Texto generado: {}".format(generated_text))
        print()


generation_callback = LambdaCallback(on_epoch_end=on_epoch_end)  

Entrena ahora tu modelo. No te olvides de añadir *generation_callback* a la lista de callbacks utilizados en fit(). Ya que las métricas de clasificación no son tan críticas aquí (no nos importa tanto acertar el carácter exacto, sino obtener una distribución de probabilidad adecuada), no es necesario monitorizar la accuracy ni usar validation data, si bien puedes añadirlos para asegurarte de que todo está en orden.


In [ ]:
n_epochs  = 300
batch_size = 32

model.fit(X, y, batch_size=batch_size, epochs=n_epochs, callbacks=[generation_callback])

Epoch 1/300
50000/50000 [==============================] - 20s 402us/step - loss: 1.6488 - categorical_accuracy: 0.4878




------> Epoch: 1 - Generando texto con temperature 0.2
Seed: aljaba y saetas; éste, vestido
Texto generado: aljaba y saetas; éste, vestido de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la

------> Epoch: 1 - Generando texto con temperature 0.5
Seed: aljaba y saetas; éste, vestido
Texto generado: aljaba y saetas; éste, vestido de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de la primero de 

Seed: cona, es la mesma de
don gaife
Texto generado: cona, es la mesma de
don gaifero, y así, que es la vida a los delante de los primeros de sus manos sancho en la caballero a los dellas de las parecia y sancho en el almana de la mis de las deligues de las parecia y los teminas y la misar de las parecia y esta en el caballero de la maleras de las parecia y esta en el caballero de

------> Epoch: 5 - Generando texto con temperature 1.2
Seed: cona, es la mesma de
don gaife
Texto generado: cona, es la mesma de
don gaifero, y así, que es la vida a los delante de los primeros de sus manos sancho en la caballero a los dellas de las parecia y sancho en el almana de la mis de las deligues de las parecia y los teminas y la misar de las parecia y esta en el caballero de la maleras de las parecia y esta en el caballero de

Epoch 6/300
50000/50000 [==============================] - 23s 467us/step - loss: 1.5092 - categorical_accuracy: 0.5296




------> Epoch: 6 - Generando texto con temperature 

50000/50000 [==============================] - 23s 469us/step - loss: 1.4064 - categorical_accuracy: 0.5596




------> Epoch: 10 - Generando texto con temperature 0.2
Seed: re represente más impropiedade
Texto generado: re represente más impropiedade a la suces, y así, que le de manera de la sus del desto de la mendad en las dellas de sus manos que la puertad de sus manos y del presponder en el caballería de la mis de su amo en el arrendió que le dijo en el arreció don quijote y sancho panza en la mis de su amo en el arrendió que le dijo en el 

------> Epoch: 10 - Generando texto con temperature 0.5
Seed: re represente más impropiedade
Texto generado: re represente más impropiedade a la suces, y así, que le de manera de la sus del desto de la mendad en las dellas de sus manos que la puertad de sus manos y del presponder en el caballería de la mis de su amo en el arrendió que le dijo en el arreció don quijote y sancho panza en la mis de su amo en el arrendió que le dijo en el 

------

Seed: ho, y luego, con una voz
ronqu
Texto generado: ho, y luego, con una voz
ronque al anter lo que de allís de las manos y a los en el senos min abencias y aquellos que las manos y a las menos aligenos, y que es la en min antende a la señor que se le dijo:

-¿qué en la cosa de la cual con la mis a las porsantes y que le disto el cual de la sus deseos y de aquella menos de las pa

------> Epoch: 14 - Generando texto con temperature 1.2
Seed: ho, y luego, con una voz
ronqu
Texto generado: ho, y luego, con una voz
ronque al anter lo que de allís de las manos y a los en el senos min abencias y aquellos que las manos y a las menos aligenos, y que es la en min antende a la señor que se le dijo:

-¿qué en la cosa de la cual con la mis a las porsantes y que le disto el cual de la sus deseos y de aquella menos de las pa

Epoch 15/300
50000/50000 [==============================] - 23s 468us/step - loss: 1.2753 - categorical_accuracy: 0.6006




------> Epoch: 15 - Generando texto con temperatu

50000/50000 [==============================] - 23s 467us/step - loss: 1.1730 - categorical_accuracy: 0.6342




------> Epoch: 19 - Generando texto con temperature 0.2
Seed: pués de caído. a la sombra del
Texto generado: pués de caído. a la sombra del cabinado, y esta en estos contino de las partas, que las manos con la cosas de sus manos y así, que tenía que la pudro a la cuatria, el casa lo sie esto de cabelleros de su barrecia de la sus añor al aguntería de la misma en el ahora de la pida comedió a su esto de aquel premento de la podía lotros

------> Epoch: 19 - Generando texto con temperature 0.5
Seed: pués de caído. a la sombra del
Texto generado: pués de caído. a la sombra del cabinado, y esta en estos contino de las partas, que las manos con la cosas de sus manos y así, que tenía que la pudro a la cuatria, el casa lo sie esto de cabelleros de su barrecia de la sus añor al aguntería de la misma en el ahora de la pida comedió a su esto de aquel premento de la podía lotros

------

...

Seed: 
   que a tanto se haya estend
Texto generado: 
   que a tanto se haya estendir que peda
porqueda de un buena, si el verdimor a dos espoción de mostrabas por
consabín muy orasión palar y de suscruedos para
esta caballar me il trabala, que se todo le estrañasto bien conecha hanza; y éstra la pida solice está hiciose era lavíallí; y por lo sueres, si quijote, sino porándo a es

Epoch 145/300
50000/50000 [==============================] - 26s 511us/step - loss: 0.3923 - categorical_accuracy: 0.8698




------> Epoch: 145 - Generando texto con temperature 0.2
Seed: d de señor". digo esto porque

Texto generado: d de señor". digo esto porque
no se dijo:

-¿quiéne se la ha le sonder -dijo se pesvir una la maneste cincan illado,
maligo señora de hambre, ni andel de sen cóule, autieron es pincir a dar, y vuesto, adonuel se sancho en estos
muchos nicieles, de la sustaba nos encantado estinga del jubiaron que de
don quijote horla. ofino mali

------> Epoch: 145 - Generando texto con temper

Seed: ó la bendición,
diciendo:

-ag
Texto generado: ó la bendición,
diciendo:

-aguará en esto, dí con otres, y cuatro
a toda jenturbaciano. y que, acaborásente la menos mermo que el vuestra por ver por mir fuchar aquel
sendabra, amon torgosa quiveo, y lo mostrese. puss el cual viste cioros, no haña don funcherle.

escude
conirdo hacellera que no vuestra merced antistar por
razón

------> Epoch: 149 - Generando texto con temperature 0.5
Seed: ó la bendición,
diciendo:

-ag
Texto generado: ó la bendición,
diciendo:

-aguará en esto, dí con otres, y cuatro
a toda jenturbaciano. y que, acaborásente la menos mermo que el vuestra por ver por mir fuchar aquel
sendabra, amon torgosa quiveo, y lo mostrese. puss el cual viste cioros, no haña don funcherle.

escude
conirdo hacellera que no vuestra merced antistar por
razón

------> Epoch: 149 - Generando texto con temperature 1.0
Seed: ó la bendición,
diciendo:

-ag
Texto generado: ó la bendición,
diciendo:

-aguará en esto, dí con otres, y cua

Seed: pondió el castellano-, que aco
Texto generado: pondió el castellano-, que acomendo se le das, por la ansancias, y de asgúnguntan seños, hay mal llevado que pesa
menestera, sin malero, sin que ton racagión, sin vuestra iba del dicio, y hacían vuestra vesir en estas
manos dijo que imaricia, que con mediaran el más y de abrentos que se me dar a señar lo que en verda de los posm

------> Epoch: 153 - Generando texto con temperature 1.2
Seed: pondió el castellano-, que aco
Texto generado: pondió el castellano-, que acomendo se le das, por la ansancias, y de asgúnguntan seños, hay mal llevado que pesa
menestera, sin malero, sin que ton racagión, sin vuestra iba del dicio, y hacían vuestra vesir en estas
manos dijo que imaricia, que con mediaran el más y de abrentos que se me dar a señar lo que en verda de los posm

Epoch 154/300
50000/50000 [==============================] - 25s 501us/step - loss: 0.3839 - categorical_accuracy: 0.8736s - loss:




------> Epoch: 154 - Generando texto c

50000/50000 [==============================] - 25s 507us/step - loss: 0.3631 - categorical_accuracy: 0.8782




------> Epoch: 158 - Generando texto con temperature 0.2
Seed: sobajada, digo, del soberana s
Texto generado: sobajada, digo, del soberana sazcho, y, le tenitaba, dijo que lo puerbar y de aquellos que en mosera, sin judó la demiente y penándezado, con los poque es
ajospo, que fuere asióle en el buen dos branos, llamando labría y costricia del
barba con consigue y
muestra animante sancho, cua
todara en elantes y pedeso si menenter a la b

------> Epoch: 158 - Generando texto con temperature 0.5
Seed: sobajada, digo, del soberana s
Texto generado: sobajada, digo, del soberana sazcho, y, le tenitaba, dijo que lo puerbar y de aquellos que en mosera, sin judó la demiente y penándezado, con los poque es
ajospo, que fuere asióle en el buen dos branos, llamando labría y costricia del
barba con consigue y
muestra animante sancho, cua
todara en elantes y pedeso si menenter a la b

----

Seed:  padre, por no declarar el mío
Texto generado:  padre, por no declarar el mío, y, ellvimas, quezosancemos y otente, no hay alma de otra merocua, porque no habéiso, como allas, y lo que me valerados del pestriose la coliciaros del ofmero. lo que lo ha le esta de la
pider dolir lo que mi ambenis de las porminas ya la punderas delante y le rigurta y así muelos contados casalos 

------> Epoch: 162 - Generando texto con temperature 1.2
Seed:  padre, por no declarar el mío
Texto generado:  padre, por no declarar el mío, y, ellvimas, quezosancemos y otente, no hay alma de otra merocua, porque no habéiso, como allas, y lo que me valerados del pestriose la coliciaros del ofmero. lo que lo ha le esta de la
pider dolir lo que mi ambenis de las porminas ya la punderas delante y le rigurta y así muelos contados casalos 

Epoch 163/300
50000/50000 [==============================] - 25s 502us/step - loss: 0.3786 - categorical_accuracy: 0.8730




------> Epoch: 163 - Generando texto con temper

In [12]:
model.load_weights('modelo.kmodel')

## Entregable

Completa los apartados anteriores para entrenar modelos del lenguaje que sean capaces de generar texto con cierto sentido. Comentar los resultados obtenidos y cómo el modelo va mejorando época a época. Comentar las diferencias apreciadas al utilizar diferentes valores de temperatura. Entregar al menos la salida de un entrenamiento completo con los textos generados época a época.

El objetivo no es conseguir generar pasajes literarios con coherencia, sino obtener lenguaje que se asemeje en cierta manera a lo visto en el texto original y donde las palabras sean reconocibles como construcciones en castellano. Como ejemplo de lo que se puede conseguir, este es el resultado de generar texto después de 10 epochs y con temperature 0.2:


```
-----> Epoch: 10 - Generando texto con temperature 0.2
Seed: o le cautivaron y rindieron el
Texto generado: o le cautivaron y rindieron el caballero de la caballería de la mano de la caballería del cual se le dijo:

-¿quién es el verdad de la caballería de la caballería de la caballería de la caballería de la caballería, y me ha de habían de la mano que el caballero de la mano de la caballería. y que no se le habían de la mano de la c

```

Asimismo, se proponen los siguientes aspectos opcionales para conseguir nota extra:

*   Experimentar con los textos de teatro en verso de Calderón de la Barca (¿es capaz el modelo de aprender las estructuras del teatro en verso?) o con alguno de los otros textos disponibles. También se puede probar con textos de vuestra elección.
*   Experimentar con distintos valores de SEQ_LENGTH.
*   Experimentar con los hiperparámetros del modelo o probar otro tipo de modelos como GRUs o *stacked* RNNs (RNNs apiladas).
*   Experimentar utilizando embeddings en vez de representaciones one-hot.
*   (Difícil) Entrenar un modelo secuencia a secuencia en vez de secuencia a carácter.




In [22]:
def build_GRU_model(input_shape, n_output=61, lr=1.e-3):
    model = keras.Sequential()
    model.add( keras.layers.InputLayer(input_shape=input_shape))
    

    # Al igual que en el modelo pasado creamos una layer recurrente, pero con unidades GRU.
    rec_layer = keras.layers.GRU(128, return_sequences=False)
    
    model.add( keras.layers.Dense(n_output, activation='softmax') )
    
    opt = keras.optimizers.Adam(lr=lr)
    model.compile(opt, 'categorical_crossentropy', metrics=['categorical_accuracy'])
    
    model.summary()
    return model
                  
input_shape = X.shape[1:]
n_output    = y.shape[-1]

gru_model = build_GRU_model(input_shape, n_output, lr=1e-3)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 30, 61)            3782      
Total params: 3,782
Trainable params: 3,782
Non-trainable params: 0
_________________________________________________________________


In [23]:
n_epochs  = 300
batch_size = 32

model.fit(X, y, batch_size=batch_size, epochs=n_epochs, callbacks=[generation_callback])

Epoch 1/300
50000/50000 [==============================] - 23s 459us/step - loss: 3.0840 - categorical_accuracy: 0.4150




------> Epoch: 1 - Generando texto con temperature 0.2
Seed: hecho cargo en la
vida. nunca 
Texto generado: hecho cargo en la
vida. nunca todos la mer las delidos de maliso, me haceñar la merve, aunque la esto se
dejaba de mil
mucho al con la menoré para mencibo; andando el hinero, su puesto, adora, pues esta puesto, adora, y tenero de mil
mucho al con quitado a es piga, el
mudando en la señora que la en tambra merce
pera es; a en tod

------> Epoch: 1 - Generando texto con temperature 0.5
Seed: hecho cargo en la
vida. nunca 
Texto generado: hecho cargo en la
vida. nunca todos la mer las delidos de maliso, me haceñar la merve, aunque la esto se
dejaba de mil
mucho al con la menoré para mencibo; andando el hinero, su puesto, adora, pues esta puesto, adora, y tenero de mil
mucho al con quitado a es piga, el
mudando en la señora que la en tambra merce
pera es; a en t

Seed: ; pero don quijote no se le qu
Texto generado: ; pero don quijote no se le que al mujero a su prespor a la caballero, y si le discuro de la me de su mano y contra la muyan de la mucho a su puesto de la mucho a su puesto de la mucho a su puesto de la mucho a su puesto de la mucho a su puesto de la mucho a su puesto de la mucho a su puesto de la mucho a su puesto de la mucho a

------> Epoch: 5 - Generando texto con temperature 1.2
Seed: ; pero don quijote no se le qu
Texto generado: ; pero don quijote no se le que al mujero a su prespor a la caballero, y si le discuro de la me de su mano y contra la muyan de la mucho a su puesto de la mucho a su puesto de la mucho a su puesto de la mucho a su puesto de la mucho a su puesto de la mucho a su puesto de la mucho a su puesto de la mucho a su puesto de la mucho a

Epoch 6/300
50000/50000 [==============================] - 29s 580us/step - loss: 1.5912 - categorical_accuracy: 0.5182




------> Epoch: 6 - Generando texto con temperature 

50000/50000 [==============================] - 27s 548us/step - loss: 1.3939 - categorical_accuracy: 0.5676




------> Epoch: 10 - Generando texto con temperature 0.2
Seed: icios, sólo me contento
con ad
Texto generado: icios, sólo me contento
con adondo las caballeros de mi señor don quijote de la caballero y de la caballero a la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero

------> Epoch: 10 - Generando texto con temperature 0.5
Seed: icios, sólo me contento
con ad
Texto generado: icios, sólo me contento
con adondo las caballeros de mi señor don quijote de la caballero y de la caballero a la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero de la caballero

------

Seed: ; y, así como yo le
vi, le pre
Texto generado: ; y, así como yo le
vi, le prenes de mucha habra de la mucha habra de mucho historias de su hermosa de mucha habra de mucho de la manos de mano a su amadera de la habra de mil
ciento, y no se lo de la merced lo que al malos de malos de su hermosar que la de la merced de la mucha la caballero, que en el carde los que hay puesto q

------> Epoch: 14 - Generando texto con temperature 1.2
Seed: ; y, así como yo le
vi, le pre
Texto generado: ; y, así como yo le
vi, le prenes de mucha habra de la mucha habra de mucho historias de su hermosa de mucha habra de mucho de la manos de mano a su amadera de la habra de mil
ciento, y no se lo de la merced lo que al malos de malos de su hermosar que la de la merced de la mucha la caballero, que en el carde los que hay puesto q

Epoch 15/300
50000/50000 [==============================] - 27s 535us/step - loss: 1.2277 - categorical_accuracy: 0.6180




------> Epoch: 15 - Generando texto con temperatu

50000/50000 [==============================] - 26s 525us/step - loss: 1.1180 - categorical_accuracy: 0.6511




------> Epoch: 19 - Generando texto con temperature 0.2
Seed:  costumbres, y
podría ser que 
Texto generado:  costumbres, y
podría ser que el cielo, como si es el menos de la estemas y entra
cosa del caballero, y el de la mantiesen, de aquellantes y las alas altanase, si no tengo de la casa del vino de la intencias, no pudieron la menester a todos a la manos a la menustia a sus pasales que esta en una de la menestir y no asigno de la m

------> Epoch: 19 - Generando texto con temperature 0.5
Seed:  costumbres, y
podría ser que 
Texto generado:  costumbres, y
podría ser que el cielo, como si es el menos de la estemas y entra
cosa del caballero, y el de la mantiesen, de aquellantes y las alas altanase, si no tengo de la casa del vino de la intencias, no pudieron la menester a todos a la manos a la menustia a sus pasales que esta en una de la menestir y no asigno de la m

------

...

Seed: das -añadió don quijote-,
que 
Texto generado: das -añadió don quijote-,
que ni lo que puedo con sancho panza los de las camino, y si tordó, y que pueda se lo desperdido y aventa de lo reducha las algunas de sus
pasal que esto con lo suelte le especho; y tieme en la nombre de desháiller que la lerra que tomase si
es el mesmo mestere la ver al presponura a las arres y bien; p

------> Epoch: 50 - Generando texto con temperature 1.2
Seed: das -añadió don quijote-,
que 
Texto generado: das -añadió don quijote-,
que ni lo que puedo con sancho panza los de las camino, y si tordó, y que pueda se lo desperdido y aventa de lo reducha las algunas de sus
pasal que esto con lo suelte le especho; y tieme en la nombre de desháiller que la lerra que tomase si
es el mesmo mestere la ver al presponura a las arres y bien; p

Epoch 51/300
50000/50000 [==============================] - 26s 514us/step - loss: 0.7036 - categorical_accuracy: 0.7796




------> Epoch: 51 - Generando texto con temperatu

50000/50000 [==============================] - 26s 521us/step - loss: 0.6734 - categorical_accuracy: 0.7890




------> Epoch: 55 - Generando texto con temperature 0.2
Seed:  no
tuviera necesidad de maest
Texto generado:  no
tuviera necesidad de maestras alguncuntadmen.

tora es puesto correga de lo que suele nada el cabriana de su estameño rosar de se había de caballer y
desese
suelinos.

-¡que, quisáis a lleo, la dicham de los cienten no le dijo:

-segare con lo cual de sus triblas de su casa de allí y del montera dejerladora, y que nuda a una

------> Epoch: 55 - Generando texto con temperature 0.5
Seed:  no
tuviera necesidad de maest
Texto generado:  no
tuviera necesidad de maestras alguncuntadmen.

tora es puesto correga de lo que suele nada el cabriana de su estameño rosar de se había de caballer y
desese
suelinos.

-¡que, quisáis a lleo, la dicham de los cienten no le dijo:

-segare con lo cual de sus triblas de su casa de allí y del montera dejerladora, y que nuda a una

------

Seed: cuando se vio solo y en parte 
Texto generado: cuando se vio solo y en parte se duje de la tregué
cantoso primer sin las manos así,
las mujertas, tando a los cuatios.

estidado en estos que quisiese adrecuras indrían las molilatas, parece se le decir y despersaban las mujer
a recesa: si ne la veno a mi acogercia de sabio alguna a carre, que era castia, sin mujera a los
ruedo

------> Epoch: 59 - Generando texto con temperature 1.2
Seed: cuando se vio solo y en parte 
Texto generado: cuando se vio solo y en parte se duje de la tregué
cantoso primer sin las manos así,
las mujertas, tando a los cuatios.

estidado en estos que quisiese adrecuras indrían las molilatas, parece se le decir y despersaban las mujer
a recesa: si ne la veno a mi acogercia de sabio alguna a carre, que era castia, sin mujera a los
ruedo

Epoch 60/300
50000/50000 [==============================] - 26s 514us/step - loss: 0.6550 - categorical_accuracy: 0.7951




------> Epoch: 60 - Generando texto con temperatu

50000/50000 [==============================] - 25s 505us/step - loss: 0.6286 - categorical_accuracy: 0.8009s - loss: 0.627




------> Epoch: 64 - Generando texto con temperature 0.2
Seed: ieron, según
él había leído en
Texto generado: ieron, según
él había leído en ver más vistador, y en ellas se hubies; tal parece en la tengunta, de le dijo:

-aquen peso ésto no parelo ver a otro más pensariente en dijo,
¿no parece sancho, por señor gocudó a suelte que ellí se decir la mucha diro, como si esto merás de sus partes
a las manos a la engraba y las almas de su hu

------> Epoch: 64 - Generando texto con temperature 0.5
Seed: ieron, según
él había leído en
Texto generado: ieron, según
él había leído en ver más vistador, y en ellas se hubies; tal parece en la tengunta, de le dijo:

-aquen peso ésto no parelo ver a otro más pensariente en dijo,
¿no parece sancho, por señor gocudó a suelte que ellí se decir la mucha diro, como si esto merás de sus partes
a las manos a la engraba y las almas d

KeyboardInterrupt: 